the function func_run contains the main body of code which reads in data, solves Geo-I, and p mpc-H. it takes as an input the number of user.


In [1]:
import pandas as pd
import numpy as np
import math 
from casadi import *
from pylab import *
import os 
from datetime import datetime, timedelta
import pymap3d # python3 -m pip install pymap3d
import random
import pickle
from scipy.special import lambertw
from scipy import stats
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output, save
from bokeh.layouts import gridplot, row, column
from bokeh.models import Range1d
output_notebook() 



Loading BokehJS ...

In [2]:
nbuf = 30

# Functions to use

In [3]:
def system_update(system,x,y,b):
    dict = {'X': x, 'Y': y, 'N': b}
    system = system.append(dict, ignore_index = True).drop(index=[0]).reset_index(drop=True)
    return system

def privacy(system):
    n = system['N'].sum()
    if n == 0:
        barx = 0
        bary = 0
        priv = 0
    else:
        barx = np.dot(system['X'], system['N'])/n
        bary = np.dot(system['Y'], system['N'])/n
        #priv = np.sqrt(np.dot((system['X'] - barx)**2 + (system['Y'] - bary)**2, system['N'])/n) 
        priv = np.dot(np.sqrt((system['X'] - barx)**2 + (system['Y'] - bary)**2), system['N'])/n 
    return barx, bary, priv

def utility(x,y):
    return np.sqrt(x**2 + y**2)

def reset_system(system):
    n = len(system)
    X = np.array([0]*n)
    Y = np.array([0]*n)
    N = np.array([0]*n)

    system['X'] = X
    system['Y'] = Y
    system['N'] = N
    return system



def GeoI_LPPM(x,y,epsilon):
    theta_GeoI = random.uniform(0,2*math.pi)
    r_GeoI = -1/epsilon*(np.real(lambertw((random.random()-1)/math.exp(1),k=-1))+1);
    x_ctrl = x + r_GeoI*math.cos(theta_GeoI)
    y_ctrl = y + r_GeoI*math.sin(theta_GeoI)
    return x_ctrl, y_ctrl, r_GeoI 
    

## Symbolic functions


In [4]:
from sys import path

# Control
u = MX.sym("u")
v = MX.sym("v")
b = MX.sym("b")

# State
x = MX.sym("x",nbuf)
y = MX.sym("y",nbuf)
nx = MX.sym("nx",nbuf)

# Matrices
A = np.zeros((nbuf, nbuf))
B = np.zeros((nbuf, 1))
for i in range(nbuf-1):
    A[i,i+1] = 1

B[nbuf-1,0] = 1
A.tolist()
B.tolist()
A = DM(A)
B = DM(B)

# Dynamics
xplus = A@x + B@u
yplus = A@y + B@v
nplus = A@nx + B@b

# Discrete time dynamics function
F = Function('F', [x,u],[xplus])

#Privacity function
O = DM(np.ones((1,nbuf)))
xbar = (O@(x*nx))/((O@nx)+1e-8)
ybar = O@(y*nx)/((O@nx)+1e-8)
P = (O@( sqrt((x-xbar)**2 + (y-ybar)**2)*nx) )/((O@nx)+1e-8)
J = Function('J', [x,y,nx],[xbar, ybar, P],['x','y','nx'],['xbar', 'ybar','P'])

# Test privacy functions
x1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
y1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
nx1 = [1]*nbuf

## MPC function

In [5]:
#Defining mpc function
def solve_mpc(xs,ys,us,bs,horizon,nbuf,util_bound):
    nt = len(xs)
    x_mpc = []
    y_mpc = []
    U_mpc = []
    V_mpc = []
    barx_mpc = []
    bary_mpc = []
    priv_mpc = []
    util_mpc = []
    time_mpc = []
    X0 = np.array([0]*nbuf).tolist()
    Y0 = np.array([0]*nbuf).tolist()
    N0 = np.array([0]*nbuf).tolist()

    system_opt = pd.DataFrame()
    system_opt['X'] = X0
    system_opt['Y'] = Y0
    system_opt['N'] = N0

    for tk in range(0,nt-horizon):#nt-horizon):

        if bs[tk] == 0:
            U_mpc += [0]
            V_mpc += [0]
            x_mpc += [xs[tk]]
            y_mpc += [ys[tk]]
            system_opt = system_update(system_opt, xs[tk], ys[tk], 0)
            barxt, baryt, privt = privacy(system_opt)
            barx_mpc += [barxt] 
            bary_mpc += [baryt] 
            priv_mpc += [privt] 
            # Compute utility
            util_mpc += [-1]
            time_mpc += [0]
            #system_opt = system_update(system_opt, xs[tk], ys[tk], 0)

        else:
            time_0= time.time()
            X0 = system_opt['X'].tolist()
            Y0 = system_opt['Y'].tolist()
            N0 = system_opt['N'].tolist()

            # Initial conditions
            Jk = 0
            w=[]
            w0 = []
            lbw = []
            ubw = []
            g=[]
            lbg = []
            ubg = []

            # Constraints on input
            U = MX.sym("U",1)
            V = MX.sym("V",1)
            w += [U, V]
            lbw += [-inf]*2
            ubw += [inf]*2
            w0 += [0,util_bound[tk]]
            g   += [U**2 + V**2]
            lbg += [0]
            ubg += [util_bound[tk]**2]

            # Initial conditions
            Xk = MX.sym("Xk",nbuf)
            w += [Xk]
            lbw += X0
            ubw += X0
            w0 += X0
            Yk = MX.sym("Yk",nbuf)
            w += [Yk]
            lbw += Y0
            ubw += Y0
            w0 += Y0
            Nk = MX.sym("Nk",nbuf)
            w += [Nk]
            lbw += N0
            ubw += N0
            w0 += N0

            # Future values of x, y and n
            Xf=np.zeros(horizon)
            Yf=np.zeros(horizon)
            Nf=bs[tk]*np.ones(horizon)
            
            Xf[0]=xs[tk]
            Yf[0]=ys[tk]
            ###
            
            system_aux=system_opt.copy()
            system_aux=system_update(system_aux,Xf[0],Yf[0],Nf[0])
            for kk in range(1,horizon):
                Xf[kk],Yf[kk],p1=privacy(system_aux.iloc[1:]) #comment when using same position
                system_aux=system_update(system_aux,Xf[kk],Yf[kk],Nf[kk])
            

            Xf = Xf[::].tolist()
            Yf = Yf[::].tolist()
            Nf = Nf[::].tolist()
            
            # Apply the first input 
            Xk = F(Xk,Xf[0]+U)
            Yk = F(Yk,Yf[0]+V)
            Nk = F(Nk,Nf[0])
            Jt = J(x=Xk,y=Yk,nx=Nk)
            Jk += Jt['P']
            for k in range(horizon-1):
                # New NLP variable for the control
                Uk = MX.sym('U_' + str(k))
                Vk = MX.sym('V_' + str(k))
                w   += [Uk, Vk]
                lbw += [-inf]*2
                ubw += [inf]*2
                w0 += [0]*2

                g   += [Uk**2 + Vk**2]
                lbg += [0]
                ubg += [util_bound[tk+k+1]**2]

                Xk = F(Xk,Xf[k+1]+Uk)
                Yk = F(Yk,Yf[k+1]+Vk)
                Nk = F(Nk,Nf[k+1])
                Jt = J(x=Xk,y=Yk,nx=Nk)
                Jk += Jt['P']


            prob = {'f': -Jk, 'x': vertcat(*w), 'g': vertcat(*g)}
            solver = nlpsol('solver', 'ipopt', prob,{"ipopt": {"print_level": 5}});
            sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
            w_opt = sol['x'].full().flatten()
            J_opt = -sol['f']

            # Plot the solution
            u_opt = w_opt[0]
            v_opt = w_opt[1]
            U_mpc += [u_opt]
            V_mpc += [v_opt]
            x_mpc += [Xf[0] + u_opt]
            y_mpc += [Yf[0] + v_opt]
            system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)
            barxt, baryt, privt = privacy(system_opt)
            barx_mpc += [barxt] 
            bary_mpc += [baryt] 
            priv_mpc += [privt] 
            # Compute utility
            util_mpc += [utility(u_opt,v_opt)]
            time_mpc += [time.time()-time_0]
            #system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)

    clear_output(wait=False)

    data_mpc = pd.DataFrame()
    data_mpc['x'] = x_mpc
    data_mpc['y'] = y_mpc
    data_mpc['u_enu']= us[:len(x_mpc)]
    data_mpc['U'] = U_mpc
    data_mpc['V'] = V_mpc
    data_mpc['barx'] = barx_mpc
    data_mpc['bary'] = bary_mpc
    data_mpc['priv'] = priv_mpc
    data_mpc['util'] = util_mpc
    data_mpc['time'] = time_mpc

    return data_mpc#U_mpc, V_mpc, x_mpc, y_mpc, barx_mpc, bary_mpc, priv_mpc, util_mpc 

## Main function

In [6]:
def func_run(user_n,T_i,T_f,horizons,GeoI_filename):
    user= str(user_n) # Examples: abboip oilrag 51 90 14
    #Name your files
    local_url="datasets/"+dataset+"-tree/"+user+".csv"
    folder_sol='WC_20000'
    sol_file='solutions/'+dataset+'_10s/'+folder_sol+'/sol_' +str(user_n)
    file_GeoI='solutions/'+dataset+'_10s/'+GeoI_filename+'/sol_' +str(user_n)
    exists = os.path.isfile(file_GeoI+'/Data_GeoI.csv')
    if exists:
        data = pd.read_csv(local_url,names=["latitude","longitude","timestamp"])
        print("uploaded from local file")
    else:
        return #if user doesn't exist return out of the function in order to get another user
        

    if not os.path.exists(sol_file):
        os.makedirs(sol_file)
        if not os.path.isfile('solutions/'+dataset+'_10s/'+folder_sol+'/info.txt'):
            text_file = open('solutions/'+dataset+'_10s/'+folder_sol+'/info.txt', "w")
            ox = text_file.write('Geo folder used : '+GeoI_filename)
            text_file.close()
        
        
    Data_GeoI=pd.read_csv(file_GeoI+'/Data_GeoI.csv')    
    Data_real=pd.read_csv(file_GeoI+'/Data_real.csv')
    xs=Data_real['x']
    ys=Data_real['y'] 
    bs=Data_real['b']
    us=Data_real['u_enu']
    util=Data_GeoI['util'] 
    util2=Data_GeoI.loc[Data_GeoI['util']>0].mean()
    
    start_time = time.monotonic()
    Data_MPC = {} # Empty dictionary
    for h in horizons:
        data_mpc = solve_mpc(xs,ys,us,bs,h,nbuf,util)
        Data_MPC[h] = data_mpc
        end_time = time.monotonic()
        print('user',user,' horizon ',h,' runtime ',timedelta(seconds=end_time - start_time), ' total time ',timedelta(seconds=end_time - time_0))
        Data_MPC[h].to_csv(sol_file+'\\save_data_mpc_horizon_' + str(h) + '.csv')


    end_time = time.monotonic()
    print(timedelta(seconds=end_time - start_time))



    # Save data MPC
    pickle.dump( horizons, open( sol_file+"/Horizons_MPC.pkl", "wb" ) )
    pd.DataFrame(horizons).to_csv(sol_file+"/horizons.csv") 
    pickle.dump( Data_MPC, open( sol_file+"/Data_MPC.pkl", "wb" ) )
    

## Run MPC

In [7]:
dataset="privamov" # OPTIONS ARE: cabspotting privamov
#horizon 7->3min
hor=[1,2,3,5,8,13,16]#[1,2,3,4,5,6,7]#,8,9,10]
time_0 = time.monotonic()
for user in range(1,8):
    func_run(user,0,20000,hor,'GeoI_20000')

user 7  horizon  16  runtime  1 day, 3:18:52.141000  total time  3 days, 19:34:07.453000
1 day, 3:18:52.188000
